In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
# Имитируем запрос с обычного браузера
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.93 Safari/537.36'
}

In [ ]:
# Списки для хранения данных
products = [] # Продукт
weights = [] # Вес
compositions = [] # Крткий состав
calories = [] # Ккал
proteins = [] # Белки
fats = [] # Жиры
carbs = [] # Углеводы

# Последняя страница на сайте
LAST_PAGE = 65

for page in range(1, LAST_PAGE + 1):
    # Настройка URL
    url = f"https://vkusvill.ru/goods/gotovaya-eda/?PAGEN_1={page}"
    response = requests.get(url, headers=headers)

    # Проверка успешности запроса
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Получаем карточки товара
        items = soup.find_all('div', class_='ProductCards__item ProductCards__item--col-lg-1-3')

        for item in items:
            # Наименование продукции
            name = item.find('a', class_='ProductCard__link').text.strip()
            products.append(name)

            # Вес упаковки
            weight = item.find('div', class_='ProductCard__weight rtext _desktop-sm gray500 nobr').text.strip()
            weights.append(weight)

            # Краткий состав
            composition = item.find('span', itemprop='description').text.strip()
            compositions.append(composition) 

            # Пищевая ценность
            # Т.к информацию о пищевой ценности можно получить только перейдя на страницу продукта, формируем url продукта
            url_product = item.find('button', class_='A_Button _has-hover ProductCard__QuickView btn_text _desktop-sm b500 js-product-quickview').get('data-url')
            full_url_product = f"https://vkusvill.ru{url_product}"
            response_product = requests.get(full_url_product, headers=headers)

            soup_product = BeautifulSoup(response_product.text, 'html.parser')
            # Получаем блок КБЖУ продукта
            items_product = soup_product.find_all('div', class_='VV23_DetailProdPageInfoDescItem__Desc VV23_DetailProdPageAccordion__Energy')

            # Если нет информации о КБЖУ записываем None и переходим к следующей итерации
            if not items_product:
                calories.append(None)
                proteins.append(None)
                fats.append(None)
                carbs.append(None)
                continue

            # В противном случаем получаем КБЖУ продукта
            for item in items_product:
                energy_items = item.find_all('div', class_='VV23_DetailProdPageAccordion__EnergyItem')
                a = {}
                for energy_item in energy_items:
                    value = energy_item.find('div', class_='VV23_DetailProdPageAccordion__EnergyValue').text.strip()
                    description = energy_item.find('div', class_='VV23_DetailProdPageAccordion__EnergyDesc').text.strip()
                    description = description.split(',')[0]
                    a[description] = value

                # Записываем резултат пищевой ценности по продукту
                calories.append(a.get('Ккал'))
                proteins.append(a.get('Белки'))
                fats.append(a.get('Жиры'))
                carbs.append(a.get('Углеводы'))
    else:
        print(f"Ошибка при загрузке страницы: {response.status_code}")

# Создание таблицы
data = {
    'name': products,
    'weight': weights,
    'short_composition': compositions,
    'kkal': calories,
    'proteins': proteins,
    'fats': fats,
    'carbs': carbs
}
# Создание DataFrame
df = pd.DataFrame(data)

Я на странице 1| Код по https://vkusvill.ru/goods/gotovaya-eda/?PAGEN_1=1 200
Я на странице 2| Код по https://vkusvill.ru/goods/gotovaya-eda/?PAGEN_1=2 200
Я на странице 3| Код по https://vkusvill.ru/goods/gotovaya-eda/?PAGEN_1=3 200
Я на странице 4| Код по https://vkusvill.ru/goods/gotovaya-eda/?PAGEN_1=4 200
Я на странице 5| Код по https://vkusvill.ru/goods/gotovaya-eda/?PAGEN_1=5 200
Я на странице 6| Код по https://vkusvill.ru/goods/gotovaya-eda/?PAGEN_1=6 200
Я на странице 7| Код по https://vkusvill.ru/goods/gotovaya-eda/?PAGEN_1=7 200
Я на странице 8| Код по https://vkusvill.ru/goods/gotovaya-eda/?PAGEN_1=8 200
Я на странице 9| Код по https://vkusvill.ru/goods/gotovaya-eda/?PAGEN_1=9 200
Я на странице 10| Код по https://vkusvill.ru/goods/gotovaya-eda/?PAGEN_1=10 200
Я на странице 11| Код по https://vkusvill.ru/goods/gotovaya-eda/?PAGEN_1=11 200
Я на странице 12| Код по https://vkusvill.ru/goods/gotovaya-eda/?PAGEN_1=12 200
Я на странице 13| Код по https://vkusvill.ru/goods/gotovay

In [6]:
df

,name,weight,short_composition,kkal,proteins,fats,carbs
0,Азу по-татарски с картофельным пюре,230 г,"Нежные кусочки говядины, тушённые в томатном с...",117,4.3,5.8,11.9
1,"Американо 0,3, кафе",300 мл,Горячий чёрный кофе из бленда арабики и робуст...,0.8,None,None,0.2
2,"Американо 0,4, кафе",400 мл,Горячий чёрный кофе из бленда арабики и робуст...,1.2,None,None,0.3
3,Баклажаны «Пармиджано»,200 г,"Мама миа, это же настоящая итальянская пармидж...",151.7,6.9,10.1,8.3
4,Баклажаны «Пармиджано» с соусом болоньезе и сы...,200 г,"Нежные баклажаны, обжаренные в хрустящей панир...",173.5,5.9,13.1,8
...,...,...,...,...,...,...,...
1543,"Эскалоп из свинины жаренный вес., СП",,"Нежная вырезка из свиной шеи, обжаренная с неб...",295,17,25,0.5
1544,Яблочно-клубничный штрудель,110 г,Свежевыпеченный штрудель с тонким слоёным тест...,322,3.7,16.4,39.9
1545,Яичница глазунья с пикантной сосиской и томатами,247 г,"Глазунья из 3 яиц, пикантные куриные сосиски с...",173,12.8,12.7,1
1546,Яичница глазунья с пикантной сосиской и томата...,315 г,Блюдо доступно только для самовывоза из офлайн...,184.94,12.19,12.26,6.46


In [10]:
df.to_excel('vkusvill_products.xlsx', index=False)